In [18]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split

/usr/local/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [19]:
songs_triplet_df = pd.read_table("/Users/varun/Desktop/sml-project/MusicRecommendationSystem/10000.txt", header=None)
songs_triplet_df.columns = ['user_id', 'song_id', 'listen_count']
songs_metadata_df =  pd.read_csv("/Users/varun/Desktop/sml-project/MusicRecommendationSystem/song_data.csv")


In [20]:
songs_triplet_df.head()

,user_id,song_id,listen_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1


In [21]:
songs = pd.merge(songs_triplet_df, songs_metadata_df.drop_duplicates(['song_id']), on="song_id", how="left")
songs.head()

,user_id,song_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999


In [25]:
songs = songs.head(10000)

In [32]:
users = songs['user_id'].unique()
unique_all_songs = songs['song_id'].unique()

In [29]:

train_data, test_data = train_test_split(songs, test_size = 0.15, random_state=0)


In [52]:
def get_user_songs(user_id, training_data):
    user_songs = training_data[training_data["user_id"] == user_id]
    user_unique_songs = list(user_songs["song_id"].unique())
    return user_unique_songs

In [53]:
def get_song_user(song_id, training_data):
    song_user = training_data[training_data['song_id'] == song_id]
    song_unique_users  = list(song_user["user_id"].unique())
    return song_unique_users

In [54]:
print(get_song_user(unique_all_songs[0], train_data))

['b80344d063b5ccb3212f76538f3d9e43d87dca9e']


In [84]:
def create_cooccurence_matrix(user_id, training_data, unique_all_songs, user_songs):
#     user_songs = get_user_songs(user_id, training_data)
    song_users = []
    for i in range(0, len(user_songs)):
        song_users.append(get_song_user(user_songs[i], training_data))
    
    song_co_occurence_mat = np.matrix(np.zeros(shape=(len(user_songs), len(unique_all_songs))), float)
#     print (song_co_occurence_mat.shape)
    
    for i in range(0, len(unique_all_songs)):
        current_song_users = set(get_song_user(unique_all_songs[i], training_data))
#         print('i = ', i)
        for j in range(0, len(user_songs)):
#             print('j= ', j)
            j_song_users = set(song_users[j])
            common_users = current_song_users.intersection(j_song_users)
            if len(common_users) > 0:
#                 print (song_co_occurence_mat)
                all_users = current_song_users.union(j_song_users)
                score = float(len(common_users))/float(len(all_users))
                song_co_occurence_mat[j,i] = score
            else:
                song_co_occurence_mat[j,i] = 0
    return song_co_occurence_mat

In [100]:
def recommender(user_id, training_data, unique_all_songs):
    user_songs = get_user_songs(user_id, training_data)
    song_co_occurence_mat = create_cooccurence_matrix(user_id, training_data, unique_all_songs, user_songs)
    scores = song_co_occurence_mat.sum(axis = 0) / float(song_co_occurence_mat.shape[0])
    scores = np.array(scores)[0].tolist()
    
    song_sorted_indices = sorted(((v,i) for i,v in enumerate(list(scores))), reverse=True)
    output_cols = ['user_id', 'song_id', 'score']
    recommended_df = pd.DataFrame(columns=output_cols)
    score = 1
    
    for i in range(0,len(song_sorted_indices)):
        if ~np.isnan(song_sorted_indices[i][0]) and unique_all_songs[song_sorted_indices[i][1]] not in user_songs and score <= 10:
            recommended_df.loc[len(recommended_df)]=[user_id,unique_all_songs[song_sorted_indices[i][1]],song_sorted_indices[i][0]]
            score += 1
    
    return recommended_df
    

In [101]:

recommended_df = recommender(users[0], train_data, unique_all_songs)
recommended_df= pd.merge(recommended_df, songs_metadata_df, on='song_id', how='left')
recommended_df

,user_id,song_id,score,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SONIIYT12A58A7E0AF,0.041143,St. Elsewhere,Collection,Dave Grusin,1989
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOMPPAS12A8C13385B,0.041143,Misled,The Colour Of My Love,Céline Dion,1993
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOLYVAA12AF72A366B,0.041143,Oil And Water,Light Grenades,Incubus,2006
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOJWBZK12A58A78AF7,0.041143,Tiger Mountain Peasant Song,Fleet Foxes,Fleet Foxes,2008
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOJAMXH12A8C138D9B,0.041143,Meadowlarks,Fleet Foxes,Fleet Foxes,2008
5,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOFZWTX12A8C138B1D,0.041143,Id Die Without You,Most Requested,P.M. Dawn,0
6,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOFLMUK12A8C132CDC,0.041143,Meet Virginia,Train,Train,1996
7,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOCMHGT12A8C138D8A,0.041143,Heard Them Stirring,Fleet Foxes,Fleet Foxes,2008
8,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAOJYY12A58A7B2F9,0.041143,Not For You,rearviewmirror (greatest hits 1991-2003),Pearl Jam,1994
9,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOKTGRH12A58A763DA,0.041143,All In Good Time,Twelve Tribes,Richard Souther,0


In [125]:
def evaluate(train_data, test_data):
    train_users = set(train_data['user_id'].unique())
    test_users = set(test_data['user_id'].unique())
    common_users = train_users.intersection(test_users)
    print(len(common_users))
    
    N = 1
    from collections import defaultdict
    recommendation_dict = defaultdict(int)
    for i in range(0, N):
        count = 0
        for user in common_users:
            recommended_songs = set(recommender(user, train_data, unique_all_songs)['song_id'])
            test_songs = set(test_data[test_data["user_id"] == user]['song_id'])
#             print(recommended_df, test_songs)
            train_test_hit_count = len(test_songs.intersection(recommended_songs))
            recommendation_dict[train_test_hit_count] = recommendation_dict[train_test_hit_count] + 1
            print(count)
            count += 1
    
    for k, v in recommendation_dict.items():
        print(k, " - ", v)
    
    

In [126]:
evaluate(train_data, test_data)

301
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
27